## TSLA stock closing price prediction models
#### Done using three models: Linear regression model, XGBoost regression model and a naive baseline approach for comparison

#### Importing the necessary libraries

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

#### Importing the dataset (Real historical TSLA dataset from Kaggle)

In [33]:
# Load the dataset
df = pd.read_csv(r"C:\Users\aadit\Desktop\Stock project\Tasla_Stock_Updated_V2.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df.set_index('Date')
df.head()
df.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

#### Feature engineering

In [34]:
df['Date'] = pd.to_datetime(df['Date'])
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Prev_Close'] = df['Close'].shift(1)
df['Rolling_3'] = df['Close'].rolling(window=3).mean()
df['Rolling_7'] = df['Close'].rolling(window=7).mean()
df.head(10)

,Date,Open,High,Low,Close,Volume,DayOfWeek,Month,Prev_Close,Rolling_3,Rolling_7
0,2015-01-02,14.858000,14.883333,14.217333,14.620667,71466000,4,1,NaN,NaN,NaN
1,2015-01-05,14.303333,14.433333,13.810667,14.006000,80527500,0,1,14.620667,NaN,NaN
2,2015-01-06,14.004000,14.280000,13.614000,14.085333,93928500,1,1,14.006000,14.237333,NaN
3,2015-01-07,14.223333,14.318667,13.985333,14.063333,44526000,2,1,14.085333,14.051555,NaN
4,2015-01-08,14.187333,14.253333,14.000667,14.041333,51637500,3,1,14.063333,14.063333,NaN
5,2015-01-09,13.928000,13.998667,13.664000,13.777333,70024500,4,1,14.041333,13.960666,NaN
6,2015-01-12,13.536667,13.631333,13.283333,13.480667,89254500,0,1,13.777333,13.766445,14.010667
7,2015-01-13,13.554667,13.840667,13.394000,13.616667,67159500,1,1,13.480667,13.624889,13.867238
8,2015-01-14,12.388667,13.013333,12.333333,12.846000,173278500,2,1,13.616667,13.314445,13.701524
9,2015-01-15,12.966000,13.050000,12.666667,12.791333,78247500,3,1,12.846000,13.084667,13.516667


In [35]:
df.dropna(inplace = True)
df.head()

,Date,Open,High,Low,Close,Volume,DayOfWeek,Month,Prev_Close,Rolling_3,Rolling_7
6,2015-01-12,13.536667,13.631333,13.283333,13.480667,89254500,0,1,13.777333,13.766445,14.010667
7,2015-01-13,13.554667,13.840667,13.394000,13.616667,67159500,1,1,13.480667,13.624889,13.867238
8,2015-01-14,12.388667,13.013333,12.333333,12.846000,173278500,2,1,13.616667,13.314445,13.701524
9,2015-01-15,12.966000,13.050000,12.666667,12.791333,78247500,3,1,12.846000,13.084667,13.516667
10,2015-01-16,12.713333,12.966000,12.643333,12.871333,54048000,4,1,12.791333,12.836222,13.346381


In [36]:
# Hot encoding all the month numbers and days of the week so that the model treats them as categorical data
df = pd.get_dummies(df, columns=['DayOfWeek', 'Month'], drop_first=True)
df.head()

,Date,Open,High,Low,Close,Volume,Prev_Close,Rolling_3,Rolling_7,DayOfWeek_1,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
6,2015-01-12,13.536667,13.631333,13.283333,13.480667,89254500,13.777333,13.766445,14.010667,False,...,False,False,False,False,False,False,False,False,False,False
7,2015-01-13,13.554667,13.840667,13.394000,13.616667,67159500,13.480667,13.624889,13.867238,True,...,False,False,False,False,False,False,False,False,False,False
8,2015-01-14,12.388667,13.013333,12.333333,12.846000,173278500,13.616667,13.314445,13.701524,False,...,False,False,False,False,False,False,False,False,False,False
9,2015-01-15,12.966000,13.050000,12.666667,12.791333,78247500,12.846000,13.084667,13.516667,False,...,False,False,False,False,False,False,False,False,False,False
10,2015-01-16,12.713333,12.966000,12.643333,12.871333,54048000,12.791333,12.836222,13.346381,False,...,False,False,False,False,False,False,False,False,False,False


In [37]:
X_copy = df.copy()
X = df.drop(columns= ['Close', 'Date']) # the features are all the columns other than Close and Date
y = df['Close'] # our target is the closing stock price for a certain date

#### Training the Linear regression model 

In [38]:
# designating the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
dates_test = X_copy.loc[X_test.index, 'Date']
predictor = LinearRegression()
predictor.fit(X_train, y_train)

LinearRegression()

#### Evaluating the linear regression model through metrics like MAE, MSE and R squared

In [39]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = predictor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Average price: ", y_test.mean())
print("\n")
print(f"Mean absolute error of the regression model is {mae}")
print("MAE as % of average price:", (mae / y_test.mean()) * 100, "\n")

r2 = r2_score(y_test, y_pred)
print(f"R squared error of the regression model is {r2}")
print("R squared error as % of average price:", (r2 / y_test.mean()) * 100, "\n")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error of the regression model is {mse}")
print("MSE as % of average price:", (mse / y_test.mean()) * 100, "\n")

Average price:  231.76889145636875


Mean absolute error of the regression model is 2.060675414520542
MAE as % of average price: 0.8891078529011609 

R squared error of the regression model is 0.9971028736646785
R squared error as % of average price: 0.43021428259814 

Mean squared error of the regression model is 7.563140718848727
MSE as % of average price: 3.2632251340221443 



#### Adding the predictions with the other columns

In [ ]:
dates_test = X_copy.loc[X_test.index, 'Date']

test_df = X_test.copy()

test_df['Date'] = dates_test
test_df['Actual_Close'] = y_test
test_df['Regression_Prediction'] = y_pred

cols = ['Date'] + [col for col in test_df.columns if col != 'Date']
test_df = test_df[cols]

(454, 25)

#### Training the XGBoost model (Based on Gradient boosting)

In [ ]:
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred_xgb = model.predict(X_test)

(454,)

#### Merging the predictions with the other columns

In [ ]:
X_test = X_test.loc[test_df.index]
y_test = y_test.loc[test_df.index]
test_df['Gradient_Boost_Prediction'] = y_pred_xgb

#### Evaluating the XGBoost model using metrics like MAE, MSE and R squared

In [47]:
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print("Average price: ", y_test.mean())
print("\n")
print(f"Mean absolute error of the XGBoost model is {mae_xgb}")
print("MAE as % of average price:", (mae_xgb / y_test.mean()) * 100, "\n")

r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"R squared error of the XGBoost model is {r2_xgb}")
print("R squared error as % of average price:", (r2_xgb / y_test.mean()) * 100, "\n")

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f"Mean squared error of the XGBoost model is {mse_xgb}")
print("MSE as % of average price:", (mse_xgb / y_test.mean()) * 100, "\n")

Average price:  231.76889145636875


Mean absolute error of the XGBoost model is 4.120787649952892
MAE as % of average price: 1.777972714137369 

R squared error of the XGBoost model is 0.9888701165755304
R squared error as % of average price: 0.42666214191290136 

Mean squared error of the XGBoost model is 29.05529990092924
MSE as % of average price: 12.536324317881546 



#### Naive baseline prediction approach for comparison

In [ ]:
test_df['Baseline_Prediction'] = test_df['Actual_Close'].shift(1) # Predicts that today's closing price will the the same as yesterday's actual price
median_val = test_df['Baseline_Prediction'].median()
test_df['Baseline_Prediction'].fillna(median_val, inplace=True)

C:\Users\aadit\AppData\Local\Temp\ipykernel_10300\2019650124.py:5: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,Date,Open,High,Low,Volume,Prev_Close,Rolling_3,Rolling_7,DayOfWeek_1,DayOfWeek_2,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Actual_Close,Regression_Prediction,Gradient_Boost_Prediction,Baseline_Prediction
1820,2022-03-25,336.000000,340.600006,332.440002,62031600,337.973328,335.963338,319.800001,False,False,...,False,False,False,False,False,False,336.880005,337.098552,340.030579,236.473328
1821,2022-03-28,355.033325,365.959991,351.200012,102506100,336.880005,346.266663,330.287619,False,False,...,False,False,False,False,False,False,363.946655,364.356358,359.391632,336.880005
1822,2022-03-29,369.329987,371.589996,357.703339,73614900,363.946655,355.783335,339.534289,True,False,...,False,False,False,False,False,False,366.523346,359.351636,365.401215,363.946655
1823,2022-03-30,363.723328,371.316681,361.333344,59865000,366.523346,365.044444,347.764287,False,True,...,False,False,False,False,False,False,364.663330,368.025126,365.659424,366.523346
1824,2022-03-31,364.856659,367.713318,358.880005,48992700,364.663330,363.462229,351.746194,False,False,...,False,False,False,False,False,False,359.200012,362.956704,365.056976,364.663330


#### Evaluating the baseline approach using MSE, MAE and R squared

In [44]:
mae_baseline = mean_absolute_error(test_df['Actual_Close'], test_df['Baseline_Prediction'])

print("Average price predicted by baseline algorithm: ", test_df['Baseline_Prediction'].mean(), "\n")
print(f"Mean absolute error of baseline prediction: {mae}")
print("MAE as % of average price:", (mae_baseline / y_test.mean()) * 100, "\n")

mse_baseline = mean_squared_error(test_df['Actual_Close'], test_df['Baseline_Prediction'])

print(f"Mean squared error of baseline prediction: {mse}")
print("MSE as % of average price:", (mse_baseline / y_test.mean()) * 100, "\n")

r2_baseline = r2_score(test_df['Actual_Close'], test_df['Baseline_Prediction'])

print(f"R squared error of the baseline prediction is {r2}")
print("R squared error as % of average price:", (r2 / y_test.mean()) * 100, "\n")

Average price predicted by baseline algorithm:  231.80537454882383 

Mean absolute error of baseline prediction: 2.060675414520542
MAE as % of average price: 2.8043312468456723 

Mean squared error of baseline prediction: 7.563140718848727
MSE as % of average price: 40.97429027831628 

R squared error of the baseline prediction is 0.9971028736646785
R squared error as % of average price: 0.43021428259814 



#### Interactive line chart showing actual closing price, regression prediction, XGBoost prediction and baseline approach prediction

In [ ]:
import plotly.express as px
import pandas as pd

# Make sure Date is datetime for better x-axis handling
test_df['Date'] = pd.to_datetime(test_df['Date'])

fig = px.line(
    test_df,
    x='Date',
    y=['Actual_Close', 'Regression_Prediction', 'Gradient_Boost_Prediction', 'Baseline_Prediction'],
    title='Actual vs Predicted TSLA Stock closing Prices',
    labels={'value': 'Stock Price', 'variable': 'Legend'},
    template = 'plotly_dark'
)

fig.update_traces(mode='lines')  # optional: show points too
fig.update_layout(
    hovermode='x unified',
    legend_title_text='Price Type',
    xaxis_title='Date',
    yaxis_title='Price',
)


#### Interactive line chart showing errors of all the three models

In [48]:
test_df['Regression error'] = test_df['Actual_Close'] - test_df['Regression_Prediction']
test_df['Baseline error'] = test_df['Actual_Close'] - test_df['Baseline_Prediction']
test_df['Gradient boost error'] = test_df['Actual_Close'] - test_df['Gradient_Boost_Prediction']
test_df.head()
fig2 = px.line(test_df, test_df['Date'], [test_df['Regression error'], test_df['Gradient boost error'], test_df['Baseline error']], template='plotly_dark')
fig2.update_traces(mode='lines')
fig2.update_layout(
    title="Model prediction errors over time",
    hovermode='x unified',
    legend_title = "Model type",
    xaxis_title='Date',
    yaxis_title='Error (Actual price - Predicted price)',
)